# Paper Notebook (literally, but also not)

**Note:** I removed **`Patterns`** and **`Groups`**. Now the only class is **`Frame`**, plus the patterns.

But before anything else, after having checked that the kernel you're running is a Python 3 kernel (it should say Python 3 on the top right, or see Help | About), open up the Git Bash prompt, and do the usual

```
git pull
```

This should pull everythig from GitHub. Then you're ready to run this notebook.

In [27]:
from framenet.data.annotation import get_frame_df, Frame, to_layers, gf_or_target, Groups
from framenet.data.pattern    import *
import pandas as pd, qgrid

## First, we create  a `Frame` instance

A `Frame` simply groups related information w.r.t. a specific frame. 

In [28]:
# No Patterns anymore
cfm = Frame('Cause_fluidic_motion')

These lexical units have already been built.


Here is how we check the kind of `LU`s the DataFrame we just created contains: 

In [29]:
cfm.groups.data_frame['annotationSet.LU'].unique()

array(['dribble.v', 'spill.v', 'spatter.v', 'splatter.v', 'splash.v',
       'spray.v', 'squirt.v', 'pump.v'], dtype=object)

## Second: we create the symbols for all our patterns

We need to do this only once per Notebook. We need those becuase we describe the patterns l=we lokk for as Python expressions. The following applies the constructor `Lit` to the strings `'Ext'`, `'Obj'`, `'Dep'`, and `'_'` (our wildcard) and assigns the result to the corresponding Python variables. This way `Ext`, `Obj`, `Dep` and `x` can be combined in the patterns below.

In [30]:
Ext, Obj, Dep, x, CNI, DNI, INI = map(Lit, 'Ext Obj Dep _ CNI DNI INI'.split())

This is *almost* $Pattern_1$ we used in the paper. The **`&`** should be intepreted as "followed by". So `p1` is `Ext` followed by `x` (the wildcard, which matches any *single* element), followd by `Obj`, followed by `Dep`. 

In [31]:
p1 = Ext & x & Obj & Dep

### To be really precise

$Pattern_1$ was **`Ext` `_` `Obj` `Dep` `+`**, that is, like the following:

In [32]:
p1_actually = Ext & x & Obj & Many1(Dep)

So, `Obj` was followed by one or more `Dep`s. 

### To summarize:

<table>
<tr><th>The operator</th><th>means</th></tr>                       
<tr><td>&amp;</td><td>_followed by_</td></tr>
<tr><td>`|`</td><td>_one or the other_</td></tr>
<tr><td>`Many1(...)`</td><td>_one or more of ..._</td></tr>
<tr><td>`Many(...)`</td><td>_zero or more of ..._</td></tr>
<tr><td>`Opt(...)`</td><td>_optionally ..._</td></tr>
</table>


So, if we wanted to say that `x` should be followed by one or more of `Obj` or `Dep`, we would say: 

In [33]:
weird_p = x & Many1(Obj | Dep)

Which would match things like 

    v → Obj → Dep → Obj → Obj → Dep → Dep → Dep → Obj ...

### $Pattern_2$, which was **`Ext` `_` `Dep` `+`** is like this:

In [34]:
p2 =  Ext & x & Many1(Dep)

## Third: now we can actually look at patterns

This is with `p1`. The `display` method understands the following keywords, all optional:

* `pattern_matcher`: defined as the above expressions, or `None` / unspecified 
* `min_count`: the minimum count for sentences matching a pattern; a nonnegative integer number
* `negative`: a boolean specifying whether a pattern should be inverted (sentences picked when they *do not* match)
* `collapse_sentences`: a boolean that eliminates the sentences from the table (when se to `True`

In [35]:
cfm.display(pattern_matcher = p1, min_count = 0, negative=False)

In [36]:
cfm.display(min_count=2)

## Fifth: flow diagrams!

The method `select` takes a pattern (as described above), and `diagram` takes an optional keyword argument to select noncore `FE`s. The default value is `True`.

In [38]:
cfm.select(p1_actually, negative=True).diagram(noncore=True)

Written 77 records.


<IPython.core.display.Javascript object>

## More examples we worked on

### The `Filling` frame

Comment here.

In [39]:
filling = Frame('Filling')
filling.display(pattern_matcher=p1, min_count=2, negative=True)

In [40]:
filling.select(p1).diagram(noncore=True)

Written 7 records.


<IPython.core.display.Javascript object>

In [41]:
cfm.display(pattern_matcher=p1)

In [42]:
cfm.select(p1_actually).diagram(noncore=False)

Written 13 records.


<IPython.core.display.Javascript object>

In [43]:
cfm.select(p1_actually).diagram(noncore=False)

Written 13 records.


<IPython.core.display.Javascript object>

In [44]:
cm = Frame('Cause_motion')
cm.display(
    pattern_matcher=(p1_actually & Many(CNI|DNI|INI)) | (p2 & Many(CNI|DNI|INI)), 
    negative=True, 
    min_count=1,
    include_sentences=False
)

,freq.,Patterns
1,18,"Ext: Agent (NP) → v → Dep: Goal (PP) → Obj: Theme (NP)Dexter dragged over a chair and sat down at the desk .` She bit me last night , "" he said , dragging up his trouser-leg and showing her the line of tiny red marks .Finn flung down his paintbrush .Denis flung down his gun in disgust .Helen flung down her knife .Angus and Sawney flung down their weapons .Any engine machining company should be able to bore out your head and press in the valve seat inserts ( ETC 6278 ) and recut them .They tipped the contents of one oil drum into the car and threw in a burning rag .If it looks good he throws on more wood , if it looks bad he throws on more paraffin .He threw down his goose quill and grasped the clerk 's hands , pumping them up and down vigorously .The PLAYER tosses down his last coin by way of paying up , and turns away .He broke eye-contact , lifting his glass to his mouth , throwing his head back and tossing down the contents of the glass with one smooth action .Women leaning over balconies looked and tossed down flowers , younger women openly showed their approval with a strange clicking sound and Fernando walked on , oblivious of it all .And God says the wicked are like the tossing sea , for it can not be quiet , and it 's waters toss up refuse and mud .Together , by getting their fingers in the rat-holes , they managed to tug up a few feet of rotten chestnut boarding , tindery at the edges .He virtually threw me on the delivery bed and the midwife yanked down my trousers -- and there was the baby , with his head and a hand out , gazing at us .`` Noriega has closed every independent newspaper , radio and television station and arrested , tortured or forced into exile a long list of reporters , '' the statement declared .Among the ponds is one inhabited by turtles : Visitors toss in coins in the hope of bouncing one off a turtle 's head , a sure way of achieving good fortune ."
2,11,"v → Obj: Theme (NP) → Dep: Goal (PP) → CNI: Agent (None)Odd-Knut makes coffee using the method we have become familiar with -- chuck the coffee in a pot , add snow to taste , boil on the stove , when empty add coffee and/or snow as necessary .What better way to test the mettle of the new Calibra 16v than to pitch it directly against the class best , VW 's Corrado G60 ?Press each block firmly in place , and weight them with something heavy until the adhesive has set .CAPILLARY REFILL TEST : This involves pressing your thumb onto the horse 's gum .Again , change hands , push the tiller to where you were sitting , watch for the boom , as it swings across , straighten up and sit down on the new side .A TIP : If a power failure occurs ( or the mains have to be turned off ) temperature loss can be much reduced by just throwing a blanket from the bed over the aquarium .Be very careful of old fluorescent light tubes , which can explode if dropped , and never puncture old aerosol cans or throw them on a fire .Do not throw rubbish onto an open fire in the living room .Throws her bouquet to her unmarried friends before she leaves .Toss the potatoes in the butter , spoon into a warmed serving dish and garnish with more sprigs of basil .Or moving it to Lubbock or somewhere possibly is not the answer ."
3,11,"Ext: Agent (NP) → v → Dep: Source (PP) → Obj: Theme (NP)Lily turned to the mirror and dragged off her hat .Lucy paced , rampaged , flung out scads of unrequited ire , impassioned as Jay had never seen her .The American grinned and pushed out a hand , a formal gesture .He paused as he tugged off the filthy tarpaulin .He began to tug off his mask .Henry pulled open the door , yanked off the three top sheets and scuttled back to his car .` I 'll take off your shoes , "" Judy offered , yanking off his handmade Italian loafers and then tying his ankles to the bed with his beige silk socks , so that he was spread-eagled .Then she pulled the biscuit tin towards her across the table , yanked off the lid .Henry yanked out the jack , groped 

In [45]:
cm.select(p1_actually).diagram(noncore=False)

Written 22 records.


<IPython.core.display.Javascript object>

In [46]:
p1_NI = Ext & x & Obj & Many1(Dep) & Many(x)
cm.display(pattern_matcher=p1_NI, min_count=0)

In [47]:
p2 = Ext & x & Many1(Dep)
cm.select(p2).diagram(size=(960, 768))

Written 22 records.


<IPython.core.display.Javascript object>

The end.